In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=600,
    return_messages=True,
)

examples = [
    {
        "movie": "타이타닉",
        "answer": "🚢💔🌊",
    },
    {
        "movie": "스타워즈",
        "answer": "🌌🚀⚔️",
    },
    {
        "movie": "반지의 제왕",
        "answer": "💍🌋🧙‍♂️",
    },   
]

prom_Template = ChatPromptTemplate.from_messages(
    [
        ('system', "너는 세계 최고의 영화 평론가야. 하지만 아쉽게도 너는 표현의 자유를 얻지 못했어. 너는 이모티콘으로만 영화를 설명할 수 있으며, 어떠한 경우에도 3개의 이모티콘으로만 영화를 표현할 수 밖에 없어."),
        ("human", "{movie}에 대해서 설명해줘"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prom_Template,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{movie_name}"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie_name):
    result = chain.invoke({"movie_name": movie_name})
    memory.save_context(
        {"input": movie_name}, 
        {"output": result.content})
    

In [30]:
invoke_chain('부산행')

🧟‍♂️🚂🔥

In [31]:
invoke_chain('기생충')

🏠🐜💰

In [32]:
memory.load_memory_variables({})['history'][0].content 

'부산행'